In [24]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.runnables import RunnablePassthrough

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.text_splitter import SemanticChunker

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain import hub


Import pdf

In [4]:
Loader = PyPDFLoader
FILE_PATH = 'D:/AIO/Module 1/ProjectRAG/test.pdf'
loader = Loader(FILE_PATH)
documents = loader.load()
print(f"Number of Pages: {len(documents)}")

Number of Pages: 20


Create embedding model for executing Vietnamese 

In [7]:
embedding = HuggingFaceEmbeddings(
    model_name = "bkai-foundation-models/vietnamese-bi-encoder"
)

Text splitter

In [8]:
semantic_splitter = SemanticChunker(
    embeddings=embedding,
    buffer_size=1,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95,
    min_chunk_size=500,
    add_start_index=True
)

In [11]:
docs = semantic_splitter.split_documents(documents)
print('Number of semantic chunks: ', len(docs))

Number of semantic chunks:  32


Create database vector

In [13]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)
retriever = vector_db.as_retriever()

In [14]:
#test
result = retriever.invoke('What is YOLO?')
print("Number of revelant documents: ", len(result))

Number of revelant documents:  4


Use Vicuna-7b


In [16]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [18]:
# model and tokenizer
MODEL_NAME = 'lmsys/vicuna-7b-v1.5'

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config = nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.57s/it]
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [20]:
model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline
)

Device set to use cuda:0


Run

In [26]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "YOLOv10 là gì?"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()

print(answer)

c:\Users\LEGION\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


YOLOv10 là một phiên bản mới của hệ thống dự đoán đối tượng (Object Detection) được giới thiệu vào năm 2024 bởi Chien-Yao Wang, I-Hau Yeh, và Hong-Yuan Mark Liao. Mô hình này cải thiện độ chính xác và tốc độ so với YOLOv8 và giới thiệu nhiều kỹ thuật mới như Programmable Gradient Information (PGI) và Generalized Efficient Layer Aggregation Network (GELAN). YOLOv10 đạt được kết quả tốt nhất về khía cạnh Độ trễ (Latency) và Số lượng tham số mô hình (Number of parameters) trong khi vẫn giữ được độ chính xác (COCO AP) cao.
